# Modelling the Drag Coefficient using Computational Fluid Dynamics:

OpenFoam is a standalone software, documentation is available online. But beucause we need to create a 3 Dimensional grid of values to accurately account coefficient of drag values accross the regime we need to automate the process using a script. Here's a very early iteration of the same:

In [ ]:
import os
import numpy as np
import subprocess
from multiprocessing import Pool

In [ ]:
# Define ranges for orientations, velocities, and densities
orientations = np.linspace(0, 360, 10)  # degrees
velocities = np.linspace(10, 100, 10)  # m/s
densities = np.linspace(0.9, 1.3, 10)  # kg/m^3

Cd_matrix = np.zeros((len(orientations), len(velocities), len(densities)))

# Utility: Rotate STL file
def rotate_stl(input_file, output_file, angle):
    """Rotates the STL file using OpenSCAD."""
    scad_script = f"""
        rotate([0, 0, {angle}])
            import("{input_file}");
    """
    scad_file = "rotate.scad"
    with open(scad_file, "w") as f:
        f.write(scad_script)
    subprocess.run(["openscad", "-o", output_file, scad_file], check=True)

# Utility: Parse Cd from postProcess output
def parse_cd_from_output(stdout):
    """Extracts the drag coefficient (Cd) from OpenFOAM's postProcess output."""
    for line in stdout.decode().split("\n"):
        if "Cd" in line:
            try:
                return float(line.split("=")[1].strip())
            except (IndexError, ValueError):
                continue
    raise ValueError("Cd not found in postProcess output.")

# Simulation function
def run_openfoam_simulation(params):
    orientation, velocity, density, model_path = params
    case_dir = f"case_orientation_{orientation}_velocity_{velocity}_density_{density}"
    os.makedirs(case_dir, exist_ok=True)
    
    # Copy base case template
    subprocess.run(["cp", "-r", "template_case/*", case_dir], shell=True, check=True)

    # Rotate STL file for orientation
    rotated_stl = os.path.join(case_dir, "rotated_model.stl")
    rotate_stl(model_path, rotated_stl, orientation)
    
    # Modify OpenFOAM input files
    velocity_file = os.path.join(case_dir, "0/U")
    density_file = os.path.join(case_dir, "constant/transportProperties")
    
    with open(velocity_file, "w") as vf:
        vf.write(f"internalField uniform ({velocity} 0 0);\n")
    with open(density_file, "w") as df:
        df.write(f"rho [1 -3 0 0 0 0 0] {density};\n")
    
    try:
        # Run OpenFOAM commands
        subprocess.run(["blockMesh"], cwd=case_dir, check=True)
        subprocess.run(["snappyHexMesh", "-overwrite"], cwd=case_dir, check=True)
        subprocess.run(["simpleFoam"], cwd=case_dir, check=True)

        # Post-process results to extract Cd
        result = subprocess.run(["postProcess", "-func", "forces"], cwd=case_dir, capture_output=True, check=True)
        return parse_cd_from_output(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Error in OpenFOAM simulation at {case_dir}: {e}")
        return np.nan  # Return NaN to indicate a failed simulation

# Main simulation loop with parallel processing
def main():
    model_path = "model.stl"  # Input STL file
    params = [
        (orientation, velocity, density, model_path)
        for orientation in orientations
        for velocity in velocities
        for density in densities
    ]

    with Pool() as pool:
        results = pool.map(run_openfoam_simulation, params)

    # Fill the Cd_matrix
    idx = 0
    for i, orientation in enumerate(orientations):
        for j, velocity in enumerate(velocities):
            for k, density in enumerate(densities):
                Cd_matrix[i, j, k] = results[idx]
                idx += 1

    # Save results
    np.save("Cd_matrix.npy", Cd_matrix)
    print("Cd matrix saved to Cd_matrix.npy")

if __name__ == "__main__":
    main()